In [1]:
import pandas as pd
import numpy as np
import time

import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)
import ibmdata

In [49]:
# Wafer Level df

daysback = 365
starttime = time.time()
Query = """
WITH lot_name_cte AS(
    SELECT DISTINCT wafer_id, E.CELL_NAME AS Lot_Name
    FROM DMIW_SYSTEMS.WAFER A,
        DMIW_SYSTEMS.PRODUCT B,
        DMIW_SYSTEMS.EWR_EVENT_FACT C,
        DMIW_SYSTEMS.EWR_EVENT D,
        DMIW_SYSTEMS.EWR_CELL E
    WHERE A.WAFERKEY=C.WAFERKEY
        AND B.PRODUCTKEY=C.PRODUCTKEY
        AND  C.EWR_CELLKEY=E.EWR_CELLKEY
        AND OWNER_USER_ID='hai.zhu@ibm.com'
        AND B.FAMILY_CODE = 'Q5'
        AND EWR_NAME IN ('7HPP_Name_P10')
        AND CELL_ID LIKE '7HPP_Name_P10%'
        AND E.CELL_NAME NOT IN ('Lot0-1','Lot0-2','uncertain')
),
sail_cte AS(
    SELECT LEFT(wafer_id,5) AS lot_id_base, lot_id, wafer_id, family_code, MIN(tw.Last_test_date) AS Sail_date, 1 AS Sail_n,
      AVG(CASE WHEN parm_Label = 'SAILALL_H2_0p65to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAILALL_0p65to1p05
    FROM DMIW.PTileWaferFact ptwf
    INNER JOIN DMIW_SYSTEMS.TestParm tp ON ptwf.testparmkey = tp.testparmkey
    INNER JOIN DMIW_SYSTEMS.TestedWafer tw ON ptwf.testedWaferKey = tw.testedWaferKey
    WHERE tw.Last_test_date >= (current date - {} days)
      AND tw.Tech_id = '7HPP'
      AND tw.Calcdefs in ('SSL11', 'SSL21', 'SSL31')
      AND weighted_Mean IS NOT null AND ABS(weighted_Mean) < 1e25
      AND UCASE(tp.parm_Label) LIKE '%_PERFECTYIELD'
    GROUP BY lot_id, wafer_id, family_code
    ORDER BY Sail_date desc, lot_id, wafer_id
),
pele_cte AS(
    SELECT lot_Id, wafer_Id, family_code, MIN(last_Test_Date) AS Pele_date, 1 as Pele_n,
        AVG(CASE WHEN ucase(tp.parm_Label) = 'PELEALL_H2_L3_HL_PERFECTYIELD' THEN yield ELSE null END) AS PELE_ALL_L3_Vmax_PY
    FROM DMIW.PTileWaferFact ptwf
    INNER JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = ptwf.testParmKey 
    INNER JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = ptwf.testedWaferKey
    WHERE Last_test_date >= (current date - {} days) AND
        Tech_id = '7HPP' AND tw.Calcdefs IN ('PEL11', 'PEL21', 'PEL31') AND
        weighted_Mean IS NOT null AND abs(weighted_Mean) < 1e25 AND
        (ucase(tp.parm_Label) LIKE 'PELE%_H2_L3_%PERFECTYIELD')
    GROUP BY lot_Id, wafer_Id, family_code
)
SELECT ln.lot_name, s.lot_id, s.wafer_id, SAILALL_0p65to1p05, PELE_ALL_L3_Vmax_PY
FROM sail_cte s
FULL OUTER JOIN pele_cte p ON s.wafer_id = p.wafer_id
LEFT JOIN lot_name_cte ln ON s.wafer_id = ln.wafer_id
ORDER BY Sail_date desc, s.lot_id, s.wafer_id
""".format(daysback,daysback,daysback)

w_df = ibmdata.isdw.query(Query)
w_df['sailall_0p65to1p05'] = w_df['sailall_0p65to1p05'].astype(float).round(2)
w_df['pele_all_l3_vmax_py'] = w_df['pele_all_l3_vmax_py'].astype(float).round(2)

print(time.time()-starttime)

1.6529178619384766


In [23]:
PARMS = "('FIN PW', 'FC ACI CD', 'FE ACI CD', 'RX ACI CD', 'FIN BCD', 'Fin HT', 'PX ACI CD', 'PC PW', 'Final PC ACI CD (Logic CD)', 'Final PC ACI OCD_BCD', 'PC ACI CD (Logic 60CPP)', 'TK eSiGe proximity', 'TK recess depth', 'TJ ACI CD', 'TJ recess depth', 'eSiGe proximity', 'eSiGe lateral growth CD_1fin', 'eSiGe lateral growth CD_3fin', 'RG ACI CD', 'RG recess depth', 'eSD proximity', 'eSD lateral growth CD_1fin', 'eSD lateral growth CD_3fin', 'CT ACI CD1(SRAM)', 'CT OCD BCD', 'Gate ACI CD', 'IL/HK THK', 'PSA TiN THK', '12.5 1st pTiN THK', '13.0 2nd TiN THK(TP)', '13.2 3rd pTIN THK(TR)', '13.5 4th TiN THK', 'TiAlC THK', 'Metal gate height OCD', 'FMB ACI CD', 'FMC ACI CD', '14.0 CA ACI (Logic)', '14.0 CA ACI (Logic_long)', '14.0 CA ACI (SRAM)', '14.0 CA ACI (SRAM_long)', '14.5 CB ACI (Logic_Long)', '14.5 CB ACI CD(Logic)', '14.5 CB ACI CD1(SRAM)', '14.65 CM ACI (Logic)', '14.65 CM ACI (SRAM)', '14.65 CM CMP THK (SKEW)', 'BEOL_V0 HCD', 'BEOL_M1 HCD', 'BEOL_M1 height', 'BEOL_M2 HCD', 'BEOL_V1 FCD', 'BEOL_M2 height', 'BEOL_M3 HCD', 'BEOL_V2 FCD', 'BEOL_M3 height', '18.0 M4 ASI CD', '18.1 I4 ASI CD', '18.2 J4 ASI CD', 'BEOL_I4 HCD', 'BEOL_J4 HCD', 'BEOL_M4 HCD', 'BEOL_Z4 HCD', 'BEOL_V3 FCD', 'BEOL_M4 height', 'BEOL_D5 HCD', 'BEOL_MD FCD', 'BEOL_D5 height', 'BEOL_D6 HCD', 'BEOL_S5 FCD', 'BEOL_D6 height', 'BEOL_D7 HCD', 'BEOL_S6 FCD', 'BEOL_D7 height', 'BEOL_D8 HCD', 'BEOL_S7 FCD', 'BEOL_D8 height', 'BEOL_DH HCD', 'BEOL_H1 FCD', 'BEOL_H1 height', 'BEOL_N1 HCD', 'BEOL_H2 FCD', 'BEOL_H2 height', 'BEOL_N2 HCD', 'BEOL_H3 FCD', 'BEOL_H3 height', 'BEOL_N3 HCD', 'BEOL_H4 FCD', 'BEOL_H4 height', 'BEOL_HB HCD', 'BEOL_B1 FCD', 'BEOL_B1 height', 'BEOL_Y1 HCD', 'BEOL_B2 FCD', 'BEOL_B2 height', 'BEOL_BG HCD', 'BEOL_G1 FCD', 'BEOL_G1 height', 'BEOL_T1 HCD', 'BEOL_G2 FCD', 'BEOL_G2 height', 'BEOL_JQ1 FCD', 'BEOL_OI1 FCD', 'BEOL_OI1 height', 'BEOL_PM DCD', 'BEOL_QM DCD', 'BEOL_HM DCD', 'BEOL_JQ2 FCD', 'BEOL_OI2 FCD', 'BEOL_OI2 height')"

In [50]:
# Femi's parameters
starttime = time.time()
DAYSBACK = 365
QUERY = f"""
SELECT wafer_Id, dcItem_Name AS parameter, avg(dcItem_Value) AS mean 
FROM DMIW.MeasuredRawFact mrf
INNER JOIN DMIW_SYSTEMS.MeasuredParm mp ON mp.measuredParmKey = mrf.measuredParmKey
INNER JOIN DMIW_SYSTEMS.MeasuredWafer mw ON mw.measuredWaferKey = mrf.measuredWaferKey
WHERE Measurement_date >= (current date - {DAYSBACK} days) 
AND Tech_id = '7HPP'
AND meas_Pd_Id <> '' AND dcItem_Value IS NOT null
AND ucase(dcItem_Name) not in ('XCOORD', 'YCOORD', 'XSITE', 'YSITE', 'FIELD X', 'FIELD Y', 'SITE X', 'SITE Y', 'SITEID', 'ROW', 'COLUMN') 
AND dcItem_Name IN {PARMS}
GROUP BY lot_Id, wafer_Id,  dcItem_Name, meas_pd_Id
ORDER BY lot_Id, wafer_Id, dcItem_Name
"""
df = ibmdata.isdw.query(QUERY)
print(time.time()-starttime)

2.365582227706909


In [51]:
df

,wafer_id,parameter,mean
0,AZ46K-10,18.0 M4 ASI CD,100.192308
1,AZ46K-10,18.2 J4 ASI CD,98.553846
2,AZ46K-20,18.0 M4 ASI CD,106.692308
3,AZ46K-20,18.1 I4 ASI CD,102.700000
4,AZ46K-20,18.2 J4 ASI CD,105.407692
...,...,...,...
6278,AZ8N9-24,Final PC ACI OCD_BCD,93.911538
6279,AZ8N9-24,PC ACI CD (Logic 60CPP),96.350000
6280,AZ8N9-24,PC PW,23076.923077
6281,AZ8N9-13,Final PC ACI OCD_BCD,97.161538


In [52]:
table = pd.pivot_table(df, values='mean', index=['wafer_id'],columns=['parameter'], aggfunc=np.mean).reset_index()

In [53]:
table

parameter,wafer_id,12.5 1st pTiN THK,13.0 2nd TiN THK(TP),13.2 3rd pTIN THK(TR),13.5 4th TiN THK,14.0 CA ACI (Logic),14.0 CA ACI (Logic_long),14.0 CA ACI (SRAM),14.0 CA ACI (SRAM_long),14.5 CB ACI (Logic_Long),14.5 CB ACI CD(Logic),14.5 CB ACI CD1(SRAM),14.65 CM ACI (Logic),14.65 CM ACI (SRAM),14.65 CM CMP THK (SKEW),18.0 M4 ASI CD,18.1 I4 ASI CD,18.2 J4 ASI CD,BEOL_B1 FCD,BEOL_B1 height,BEOL_B2 FCD,BEOL_B2 height,BEOL_BG HCD,BEOL_D5 HCD,BEOL_D5 height,BEOL_D6 HCD,BEOL_D6 height,BEOL_D7 HCD,BEOL_D7 height,BEOL_D8 HCD,BEOL_D8 height,BEOL_DH HCD,BEOL_G1 FCD,BEOL_G1 height,BEOL_G2 FCD,BEOL_G2 height,BEOL_H1 FCD,BEOL_H1 height,BEOL_H2 FCD,BEOL_H2 height,BEOL_H3 FCD,BEOL_H3 height,BEOL_H4 FCD,BEOL_H4 height,BEOL_HB HCD,BEOL_HM DCD,BEOL_I4 HCD,BEOL_J4 HCD,BEOL_JQ1 FCD,BEOL_JQ2 FCD,BEOL_M1 HCD,BEOL_M1 height,BEOL_M2 HCD,BEOL_M2 height,BEOL_M3 HCD,BEOL_M3 height,BEOL_M4 HCD,BEOL_M4 height,BEOL_MD FCD,BEOL_N1 HCD,BEOL_N2 HCD,BEOL_N3 HCD,BEOL_OI1 FCD,BEOL_OI1 height,BEOL_OI2 FCD,BEOL_OI2 height,BEOL_PM DCD,BEOL_QM DCD,BEOL_S5 FCD,BEOL_S6 FCD,BEOL_S7 FCD,BEOL_T1 HCD,BEOL_V0 HCD,BEOL_V1 FCD,BEOL_V2 FCD,BEOL_V3 FCD,BEOL_Y1 HCD,BEOL_Z4 HCD,CT ACI CD1(SRAM),CT OCD BCD,FC ACI CD,FE ACI CD,FIN BCD,FIN PW,FMB ACI CD,FMC ACI CD,Fin HT,Final PC ACI CD (Logic CD),Final PC ACI OCD_BCD,Gate ACI CD,IL/HK THK,Metal gate height OCD,PC ACI CD (Logic 60CPP),PC PW,PSA TiN THK,PX ACI CD,RG ACI CD,RG recess depth,RX ACI CD,TJ ACI CD,TJ recess depth,TK eSiGe proximity,TK recess depth,TiAlC THK,eSD lateral growth CD_1fin,eSD lateral growth CD_3fin,eSD proximity,eSiGe lateral growth CD_1fin,eSiGe lateral growth CD_3fin,eSiGe proximity
0,AZ46K-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.192308,NaN,98.553846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AZ46K-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.692308,102.7,105.407692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AZ46Y-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.046154,NaN,101.323077,103.184615,NaN,99.761538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AZ46Y-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.161538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AZ46Y-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92.184615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [55]:
merged_df = w_df.merge(table, how='inner', on=['wafer_id'])

In [56]:
merged_df

,lot_name,lot_id,wafer_id,sailall_0p65to1p05,pele_all_l3_vmax_py,12.5 1st pTiN THK,13.0 2nd TiN THK(TP),13.2 3rd pTIN THK(TR),13.5 4th TiN THK,14.0 CA ACI (Logic),14.0 CA ACI (Logic_long),14.0 CA ACI (SRAM),14.0 CA ACI (SRAM_long),14.5 CB ACI (Logic_Long),14.5 CB ACI CD(Logic),14.5 CB ACI CD1(SRAM),14.65 CM ACI (Logic),14.65 CM ACI (SRAM),14.65 CM CMP THK (SKEW),18.0 M4 ASI CD,18.1 I4 ASI CD,18.2 J4 ASI CD,BEOL_B1 FCD,BEOL_B1 height,BEOL_B2 FCD,BEOL_B2 height,BEOL_BG HCD,BEOL_D5 HCD,BEOL_D5 height,BEOL_D6 HCD,BEOL_D6 height,BEOL_D7 HCD,BEOL_D7 height,BEOL_D8 HCD,BEOL_D8 height,BEOL_DH HCD,BEOL_G1 FCD,BEOL_G1 height,BEOL_G2 FCD,BEOL_G2 height,BEOL_H1 FCD,BEOL_H1 height,BEOL_H2 FCD,BEOL_H2 height,BEOL_H3 FCD,BEOL_H3 height,BEOL_H4 FCD,BEOL_H4 height,BEOL_HB HCD,BEOL_HM DCD,BEOL_I4 HCD,BEOL_J4 HCD,BEOL_JQ1 FCD,BEOL_JQ2 FCD,BEOL_M1 HCD,BEOL_M1 height,BEOL_M2 HCD,BEOL_M2 height,BEOL_M3 HCD,BEOL_M3 height,BEOL_M4 HCD,BEOL_M4 height,BEOL_MD FCD,BEOL_N1 HCD,BEOL_N2 HCD,BEOL_N3 HCD,BEOL_OI1 FCD,BEOL_OI1 height,BEOL_OI2 FCD,BEOL_OI2 height,BEOL_PM DCD,BEOL_QM DCD,BEOL_S5 FCD,BEOL_S6 FCD,BEOL_S7 FCD,BEOL_T1 HCD,BEOL_V0 HCD,BEOL_V1 FCD,BEOL_V2 FCD,BEOL_V3 FCD,BEOL_Y1 HCD,BEOL_Z4 HCD,CT ACI CD1(SRAM),CT OCD BCD,FC ACI CD,FE ACI CD,FIN BCD,FIN PW,FMB ACI CD,FMC ACI CD,Fin HT,Final PC ACI CD (Logic CD),Final PC ACI OCD_BCD,Gate ACI CD,IL/HK THK,Metal gate height OCD,PC ACI CD (Logic 60CPP),PC PW,PSA TiN THK,PX ACI CD,RG ACI CD,RG recess depth,RX ACI CD,TJ ACI CD,TJ recess depth,TK eSiGe proximity,TK recess depth,TiAlC THK,eSD lateral growth CD_1fin,eSD lateral growth CD_3fin,eSD proximity,eSiGe lateral growth CD_1fin,eSiGe lateral growth CD_3fin,eSiGe proximity
0,CS20B,AZ75RB.1,AZ75R-01,93.42,75.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.007143,NaN,NaN,NaN,100.385714,NaN,100.546154,NaN,NaN,105.557143,100.076923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CS20B,AZ75RB.1,AZ75R-02,90.78,94.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.307692,99.938462,100.546154,NaN,NaN,NaN,NaN,NaN,NaN,101.414286,NaN,NaN,NaN,100.071429,NaN,NaN,NaN,NaN,97.142857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CS20B,AZ75RB.1,AZ75R-03,90.78,81.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.330769,100.376923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.821429,NaN,NaN,NaN,100.221429,NaN,100.576923,NaN,NaN,103.700000,100.246154,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CS20B,AZ75RB.1,AZ75R-05,93.42,94.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.357143,NaN,NaN,NaN,100.371429,NaN,NaN,NaN,NaN,99.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.269231,99.076923,NaN
4,CS20B,AZ75RB.1,AZ75R-06,90.78,93.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [58]:
merged_df.to_csv('~/Downloads/femi.csv')

In [75]:
JustinParms = "('PCOp_ULP_60_MB_A_Rs_LT', 'PCOp_SLP_60_MB_A_Rs_LT', 'PCOp_LP_60_MB_A_Rs_LT', 'PCOp_RP_60_MB_A_Rs_LT', 'PCOp_RN_60_DB_A_Rs_LT', 'PCOp_LN_60_DB_A_Rs_LT', 'PCOp_SLN_60_DB_A_Rs_LT', 'PCOp_ULN_60_DB_A_Rs_LT', 'PC_RES_ULVTP_cpl_LT', 'PC_RES_sLVTP_cpl_LT', 'PC_RES_LVTP_cpl_LT', 'PC_RES_LVTN_cpl_LT', 'PC_RES_sLVTN_cpl_LT', 'PC_RES_ULVTN_cpl_LT', 'CA_RES_P_onRX_cpl_LT', 'ISO_NplsNpls_STI_cpl_LT', 'ISO_PplsPpls_STI_cpl_LT', 'PCOp_ULP_60_MB_A_Rs_LT', 'PCOp_SLP_60_MB_A_Rs_LT', 'PCOp_LP_60_MB_A_Rs_LT', 'PCOp_RP_60_MB_A_Rs_LT', 'PCOp_RN_60_DB_A_Rs_LT', 'PCOp_LN_60_DB_A_Rs_LT', 'PCOp_SLN_60_DB_A_Rs_LT', 'PCOp_ULN_60_DB_A_Rs_LT', 'PCOp_ULP_60_MB_B_Rs_LT', 'PCOp_SLP_60_MB_B_Rs_LT', 'PCOp_LP_60_MB_B_Rs_LT', 'PCOp_RP_60_MB_B_Rs_LT', 'PCOp_RN_60_DB_B_Rs_LT', 'PCOp_LN_60_DB_B_Rs_LT', 'PCOp_SLN_60_DB_B_Rs_LT', 'PCOp_ULN_60_DB_B_Rs_LT', 'PC_RES_ULVTP_cpl_LT', 'PC_RES_sLVTP_cpl_LT', 'PC_RES_LVTP_cpl_LT', 'PC_RES_LVTN_cpl_LT', 'PC_RES_sLVTN_cpl_LT', 'PC_RES_ULVTN_cpl_LT', 'PC2PC_FLD_No_CA_N_Lk075_cpl_LT', 'PC2PC_FLD_No_CA_P_Lk075_cpl_LT', 'PC2PC_RX_No_CA_N_Lk075_cpl_LT', 'PC2PC_RX_No_CA_P_Lk075_cpl_LT', 'EGN100GC_16FET_res_LT', 'EGN1003FGC_16FET_res_LT', 'PBLN304FGC_16FET_res_LT', 'PBLN302FGC_16FET_res_LT', 'PCSLNGC_2x2_res_LT', 'PCSLN2FGC_2x2_res_LT', 'PCS54LN2FGC_2x2_res_LT', 'PCLN2FGC_array_res_LT', 'PCRN2FGC_array_res_LT', 'EGP100GC_16FET_res_LT', 'EGP1003FGC_16FET_res_LT', 'PBLP304FGC_16FET_res_LT', 'PBLP302FGC_16FET_res_LT', 'PCSLPGC_2x2_res_LT', 'PCSLP2FGC_2x2_res_LT', 'PCS54LPGC_2x2_res_LT', 'PCSLP2FGC_array_res_LT', 'PCLP2FGC_array_res_LT', 'PCRP2FGC_array_res_LT', 'ISO_NplsNpls_STI_cpl_LT', 'ISO_PplsPpls_STI_cpl_LT', 'ISO_NplsNW_pSP_RX2W_27_wide_cpl_LT', 'ISO_PplsPW_pSP_RX2W_27_wide_cpl_LT', 'LKG_NplsPW_cpl_LT', 'LKG_PplsNW_cpl_LT', 'CT_1CPP_N_Lk075_cpl_LT', 'CT_1CPP_P_Lk075_cpl_LT', 'CT_2CPP_N_108_Lk075_cpl_M1', 'CT_2CPP_P_108_Lk075_cpl_LT', 'CT_LONG_24_N_Lk075_cpl_LT', 'CT_LONG_24_P_Lk075_cpl_LT', 'CT_1_PB150_34_N_Lk075_cpl_LT', 'CT_1_PB150_34_P_Lk075_cpl_LT', 'CAPC_M16_54PC_ULVTP_minSp7_LT', 'CAPC_M15_54PC_SLVTP_minSp7_LT', 'CAPC_M14_54PC_LVTP_minSp7_LT', 'CAPC_M13_54PC_RVTP_minSp7_LT', 'CAPC_M5n_54PC_RVTN_minSp7_LT', 'CAPC_M6n_54PC_LVTN_minSp7_LT', 'CAPC_M7n_54PC_SLVTN_minSp7_LT', 'CAPC_M8n_54PC_ULVTN_minSp7_LT', 'CAPC_M12_60PC_ULVTP_minSp7_LT', 'CAPC_M11_60PC_SLVTP_minSp7_LT', 'CAPC_M10_60PC_LVTP_minSp7_LT', 'CAPC_M09_60PC_RVTP_minSp7_LT', 'CAPC_M1n_60PC_RVTN_minSp7_LT', 'CAPC_M2n_60PC_LVTN_minSp7_LT', 'CAPC_M3n_60PC_SLVTN_minSp7_LT', 'CAPC_M4n_60PC_ULVTN_minSp7_LT', 'CAPB_M01_strap_EG70_8f4d_N_Short_abs_LT', 'CAPB_M02_strap_EG100_8f4d_N_Short_abs_LT', 'CAPB_M03_strap_EG30_8f4d_N_Short_abs_LT', 'CAPB_M04_strap_EG70_8f4d_P_Short_abs_LT', 'CAPB_M05_strap_EG100_8f4d_P_Short_abs_LT', 'CAPB_M06_strap_EG30_8f4d_P_Short_abs_LT', 'CAPB_M07_strap_60PC_4f2d_N_Short_abs_LT', 'CAPB_M08_strap_60PC_4f2d_P_Short_abs_LT', 'CA_RES_P_onRX_cpl_LT', 'OPEN_CACMM1_N_HOLE_D01_cpl_LT', 'OPEN_CACMM1_P_HOLE_D01_cpl_LT', 'Short_CA_T2T_pSP_T2T_36_cpl_LT', 'Short_CA2CB_N_T2S_CACB_31.5_cpl_LT', 'Short_CB_T2T_cSP_T2T_33_CB_78x18_cpl_LT', 'OPEN_PCCBM1_N_2A_D0X_cpl_8cl_cyld_LT', 'OPEN_PCCBM1_N_2A_D0X_cpl_8cl_cyld_LT', 'CBPC_ULVTP_Kelvin~4ptRes_1mA_LT', 'CBPC_SLVTP_Kelvin~4ptRes_1mA_LT', 'CBPC_LVTP_Kelvin~4ptRes_1mA_LT', 'CBPC_RVTP_Kelvin~4ptRes_1mA_LT', 'CBPC_RVTN_Kelvin~4ptRes_1mA_LT', 'CBPC_LVTN_Kelvin~4ptRes_1mA_LT', 'CBPC_SLVTN_Kelvin~4ptRes_1mA_LT', 'CBPC_ULVTN_Kelvin~4ptRes_1mA_LT', 'CBPCiso_SLVTP_Kelvin~4ptRes_1mA_LT', 'CBPCiso_SLVTN_Kelvin~4ptRes_1mA_LT', 'CB_RES_P_Dense_cpl_LT', 'CBPB_LVTN30_Kelvin~4ptRes_1mA_LT', 'CBPB_LVTP30_Kelvin~4ptRes_1mA_LT', 'CBPB_EGRVTN100_Kelvin~4ptRes_1mA_LT', 'CBPB_EGRVTP100_Kelvin~4ptRes_1mA_LT', 'CBPC_LVT_PPC_MDB_ACT_RperUnit_LT', 'CBPC_LVT_PPC_MDB_AUX_RperUnit_LT', 'CBPC_LVT_NPC_DDB_ACT_RperUnit_LT', 'CBPC_LVT_NPC_DDB_AUX_RperUnit_LT', 'CBPC_SLVT_PPC_MDB_ACT_RperUnit_LT', 'CBPC_SLVT_PPC_MDB_AUX_RperUnit_LT', 'CBPC_SLVT_NPC_DDB_ACT_RperUnit_LT', 'CBPC_SLVT_NPC_DDB_AUX_RperUnit_LT', 'CBPC_ULVT_PPC_MDB_ACT_RperUnit_LT', 'CBPC_ULVT_PPC_MDB_AUX_RperUnit_LT', 'CBPC_ULVT_NPC_DDB_ACT_RperUnit_LT', 'CBPC_ULVT_NPC_DDB_AUX_RperUnit_LT', 'OPEN_CACMM1_N_HOLE_D01_cpl_LT', 'OPEN_CACMM1_P_HOLE_D01_cpl_LT', 'OPEN_PCCBM1_N_2A_D0X_cpl_8cl_cyld_LT', 'CARE_MA_MDB_ULVTP_M08_RperUnit_LT', 'CARE_MA_MDB_ULVTP_M06_RperUnit_LT', 'CARE_MA_MDB_ULVTP_M04_RperUnit_LT', 'CARE_MA_MDB_ULVTP_M02_RperUnit_LT', 'CARE_MA_MDB_ULVTP_00_RperUnit_LT', 'CARE_MA_MDB_ULVTP_P02_RperUnit_LT', 'CARE_MA_MDB_ULVTP_P04_RperUnit_LT', 'CARE_MA_MDB_ULVTP_P06_RperUnit_LT', 'CARE_MA_MDB_ULVTP_P08_RperUnit_LT', 'CARE_MA_MDB_ULVTP_M08_CA_only_RperUnit_LT', 'CARE_MA_MDB_ULVTP_M06_CA_only_RperUnit_LT', 'CARE_MA_MDB_ULVTP_M04_CA_only_RperUnit_LT', 'CARE_MA_MDB_ULVTP_M02_CA_only_RperUnit_LT', 'CARE_MA_MDB_ULVTP_00_CA_only_RperUnit_LT', 'CARE_MA_MDB_ULVTP_P02_CA_only_RperUnit_LT', 'CARE_MA_MDB_ULVTP_P04_CA_only_RperUnit_LT', 'CARE_MA_MDB_ULVTP_P06_CA_only_RperUnit_LT', 'CARE_MA_MDB_ULVTP_P08_CA_only_RperUnit_LT', 'CAEPMA_X_M08_DDB_ULVTN_RperUnit_LT', 'CAEPMA_X_M06_DDB_ULVTN_RperUnit_LT', 'CAEPMA_X_M04_DDB_ULVTN_RperUnit_LT', 'CAEPMA_X_M02_DDB_ULVTN_RperUnit_LT', 'CAEPMA_X_00_DDB_ULVTN_RperUnit_LT', 'CAEPMA_X_P02_DDB_ULVTN_RperUnit_LT', 'CAEPMA_X_P04_DDB_ULVTN_RperUnit_LT', 'CAEPMA_X_P06_DDB_ULVTN_RperUnit_LT', 'CAEPMA_X_P08_DDB_ULVTN_RperUnit_LT', 'CAEPMA_X_M08_MDB_ULVTP_RperUnit_LT', 'CAEPMA_X_M06_MDB_ULVTP_RperUnit_LT', 'CAEPMA_X_M04_MDB_ULVTP_RperUnit_LT', 'CAEPMA_X_M02_MDB_ULVTP_RperUnit_LT', 'CAEPMA_X_00_MDB_ULVTP_RperUnit_LT', 'CAEPMA_X_P02_MDB_ULVTP_RperUnit_LT', 'CAEPMA_X_P04_MDB_ULVTP_RperUnit_LT', 'CAEPMA_X_P06_MDB_ULVTP_RperUnit_LT', 'CAEPMA_X_P08_MDB_ULVTP_RperUnit_LT', 'CM_RES_N_H_Dense_cpl_LT', 'QSTC_CMJ_I_PC_Short_075_cpl_LT', 'QSTC_CMJ_L_PC_Short_075_cpl_LT', 'QSTC_CMJ_T_PC_Short_075_cpl_LT', 'SR_M01_D13_ND2ND_CAyNom_Short_abs_LT', 'SR_M02_D01_ND2ND_Short_abs_LT', 'SR_M02_D05_WL2BL_Short_abs_LT', 'SR_M02_D20_WL2ND_Short_abs_LT', 'SR_M02_D21_WL2ND_CT20_Short_abs_LT', 'SR_M02_D22_WL2ND_CT24_Short_abs_LT', 'SR_M01_ND2ND_CAyMA_DM7_LT', 'SR_M01_ND2ND_CAyMA_UM7_LT', 'SR_M01_ND2ND_CAyMA_minSp7_LT', 'SR_M01_ND2ND_CAyMA_win7_LT', 'SR_M01_ND2ND_CAyMA_offset7_LT', 'SR_M02_WL2BL_CBxMA_LM7_LT', 'SR_M02_WL2BL_CBxMA_RM7_LT', 'SR_M02_WL2BL_CBxMA_minSp7_LT', 'SR_M02_WL2BL_CBxMA_win7_LT', 'SR_M02_WL2BL_CBxMA_offset7_LT', 'SR_M02_WL2BL_CByMA_DM7_LT', 'SR_M02_WL2BL_CByMA_UM7_LT', 'SR_M02_WL2BL_CByMA_minSp7_LT', 'SR_M02_WL2BL_CByMA_win7_LT', 'SR_M02_WL2BL_CByMA_offset7_LT', 'SR_V03_D06_WL2BL_CBxnom_Short_abs_LT', 'SR_V03_D15_WL2ND_CAxnom_Short_abs_LT', 'SR_V04_D06_WL2BL_CBynom_Short_abs_LT', 'SR_V04_D15_WL2BL_CAxnom_Short_abs_LT', 'SR_V03_WL2BL_CBxMA_minSp7_LT', 'SR_V03_WL2BL_CBxMA_win7_LT', 'SR_V03_WL2BL_CBxMA_offset7_LT', 'SR_V03_WL2ND_CAxMA_LM7_LT', 'SR_V03_WL2ND_CAxMA_RM7_LT', 'SR_V03_WL2ND_CAxMA_minSp7_LT', 'SR_V03_WL2ND_CAxMA_win7_LT', 'SR_V03_WL2ND_CAxMA_offset7_LT', 'SR_V04_WL2BL_CByMA_minSp7_LT', 'SR_V04_WL2BL_CByMA_win7_LT', 'SR_V04_WL2BL_CByMA_offset7_LT', 'SR_V04_WL2BL_CAxMA_LM7_LT', 'SR_V04_WL2BL_CAxMA_RM7_LT', 'SR_V04_WL2BL_CAxMA_minSp7_LT', 'SR_V04_WL2BL_CAxMA_win7_LT', 'SR_V04_WL2BL_CAxMA_offset7_LT', '7HPP_SF_FEOL_ALL_cyld_LT', '7HPP_SF_MOL_ALL_cyld_LT', 'CAPB_epiEO_ALL_cyld_LT', 'CAPC_BM01_PC_Short_12doe_cyld_LT', 'CAPC_BM01_PB_Short_10doe_cyld_LT', '7HPP_GTCHN_ALL_19doe_cyld_LT', 'PCOp_ALL_86cl_cyld_LT', '7HPP_MOL_Op_ALL_nom_69doe_cyld_LT', 'CAPC_60PC_ALL_Short_MA_ALL_nom_24cl_cyld_LT', 'CAPC_60PC_N_minSp7_LT', 'CAPC_60PC_P_minSp7_LT')"

In [77]:
# Justin's parameters
starttime = time.time()
DAYSBACK = 365
QUERY = f"""
SELECT wafer_Id, parm_Label as parameter, weighted_Mean as mean
FROM DMIW.PTileWaferFact pwf
INNER JOIN DMIW_SYSTEMS.TestParm tp ON tp.testparmkey = pwf.testparmkey
INNER JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = pwf.testedWaferKey
WHERE Last_test_date >= (current date - {DAYSBACK} days) AND Tech_id = '7HPP' 
AND weighted_Mean is not null AND abs(weighted_Mean) < 1e25 
AND tp.parm_Label IN {JustinParms}
ORDER BY wafer_Id, parm_Label 
"""
df = ibmdata.isdw.query(QUERY)
print(time.time()-starttime)

25.241921186447144


In [79]:
table = pd.pivot_table(df, values='mean', index=['wafer_id'],columns=['parameter'], aggfunc=np.mean).reset_index()

In [80]:
table

parameter,wafer_id,7HPP_GTCHN_ALL_19doe_cyld_LT,7HPP_MOL_Op_ALL_nom_69doe_cyld_LT,7HPP_SF_FEOL_ALL_cyld_LT,7HPP_SF_MOL_ALL_cyld_LT,CAEPMA_X_00_DDB_ULVTN_RperUnit_LT,CAEPMA_X_00_MDB_ULVTP_RperUnit_LT,CAEPMA_X_M02_DDB_ULVTN_RperUnit_LT,CAEPMA_X_M02_MDB_ULVTP_RperUnit_LT,CAEPMA_X_M04_DDB_ULVTN_RperUnit_LT,CAEPMA_X_M04_MDB_ULVTP_RperUnit_LT,CAEPMA_X_M06_DDB_ULVTN_RperUnit_LT,CAEPMA_X_M06_MDB_ULVTP_RperUnit_LT,CAEPMA_X_M08_DDB_ULVTN_RperUnit_LT,CAEPMA_X_M08_MDB_ULVTP_RperUnit_LT,CAEPMA_X_P02_DDB_ULVTN_RperUnit_LT,CAEPMA_X_P02_MDB_ULVTP_RperUnit_LT,CAEPMA_X_P04_DDB_ULVTN_RperUnit_LT,CAEPMA_X_P04_MDB_ULVTP_RperUnit_LT,CAEPMA_X_P06_DDB_ULVTN_RperUnit_LT,CAEPMA_X_P06_MDB_ULVTP_RperUnit_LT,CAEPMA_X_P08_DDB_ULVTN_RperUnit_LT,CAEPMA_X_P08_MDB_ULVTP_RperUnit_LT,CAPB_M01_strap_EG70_8f4d_N_Short_abs_LT,CAPB_M02_strap_EG100_8f4d_N_Short_abs_LT,CAPB_M03_strap_EG30_8f4d_N_Short_abs_LT,CAPB_M04_strap_EG70_8f4d_P_Short_abs_LT,CAPB_M05_strap_EG100_8f4d_P_Short_abs_LT,CAPB_M06_strap_EG30_8f4d_P_Short_abs_LT,CAPB_M07_strap_60PC_4f2d_N_Short_abs_LT,CAPB_M08_strap_60PC_4f2d_P_Short_abs_LT,CAPB_epiEO_ALL_cyld_LT,CAPC_60PC_ALL_Short_MA_ALL_nom_24cl_cyld_LT,CAPC_60PC_N_minSp7_LT,CAPC_60PC_P_minSp7_LT,CAPC_BM01_PB_Short_10doe_cyld_LT,CAPC_BM01_PC_Short_12doe_cyld_LT,CAPC_M09_60PC_RVTP_minSp7_LT,CAPC_M10_60PC_LVTP_minSp7_LT,CAPC_M11_60PC_SLVTP_minSp7_LT,CAPC_M12_60PC_ULVTP_minSp7_LT,CAPC_M1n_60PC_RVTN_minSp7_LT,CAPC_M2n_60PC_LVTN_minSp7_LT,CAPC_M3n_60PC_SLVTN_minSp7_LT,CAPC_M4n_60PC_ULVTN_minSp7_LT,CARE_MA_MDB_ULVTP_00_CA_only_RperUnit_LT,CARE_MA_MDB_ULVTP_00_RperUnit_LT,CARE_MA_MDB_ULVTP_M02_CA_only_RperUnit_LT,CARE_MA_MDB_ULVTP_M02_RperUnit_LT,CARE_MA_MDB_ULVTP_M04_CA_only_RperUnit_LT,CARE_MA_MDB_ULVTP_M04_RperUnit_LT,CARE_MA_MDB_ULVTP_M06_CA_only_RperUnit_LT,CARE_MA_MDB_ULVTP_M06_RperUnit_LT,CARE_MA_MDB_ULVTP_M08_CA_only_RperUnit_LT,CARE_MA_MDB_ULVTP_M08_RperUnit_LT,CARE_MA_MDB_ULVTP_P02_CA_only_RperUnit_LT,CARE_MA_MDB_ULVTP_P02_RperUnit_LT,CARE_MA_MDB_ULVTP_P04_CA_only_RperUnit_LT,CARE_MA_MDB_ULVTP_P04_RperUnit_LT,CARE_MA_MDB_ULVTP_P06_CA_only_RperUnit_LT,CARE_MA_MDB_ULVTP_P06_RperUnit_LT,CARE_MA_MDB_ULVTP_P08_CA_only_RperUnit_LT,CARE_MA_MDB_ULVTP_P08_RperUnit_LT,CA_RES_P_onRX_cpl_LT,CBPC_LVT_NPC_DDB_ACT_RperUnit_LT,CBPC_LVT_NPC_DDB_AUX_RperUnit_LT,CBPC_LVT_PPC_MDB_ACT_RperUnit_LT,CBPC_LVT_PPC_MDB_AUX_RperUnit_LT,CBPC_SLVT_NPC_DDB_ACT_RperUnit_LT,CBPC_SLVT_NPC_DDB_AUX_RperUnit_LT,CBPC_SLVT_PPC_MDB_ACT_RperUnit_LT,CBPC_SLVT_PPC_MDB_AUX_RperUnit_LT,CBPC_ULVT_NPC_DDB_ACT_RperUnit_LT,CBPC_ULVT_NPC_DDB_AUX_RperUnit_LT,CBPC_ULVT_PPC_MDB_ACT_RperUnit_LT,CBPC_ULVT_PPC_MDB_AUX_RperUnit_LT,CB_RES_P_Dense_cpl_LT,CM_RES_N_H_Dense_cpl_LT,EGN1003FGC_16FET_res_LT,EGN100GC_16FET_res_LT,EGP1003FGC_16FET_res_LT,EGP100GC_16FET_res_LT,ISO_NplsNW_pSP_RX2W_27_wide_cpl_LT,ISO_NplsNpls_STI_cpl_LT,ISO_PplsPW_pSP_RX2W_27_wide_cpl_LT,ISO_PplsPpls_STI_cpl_LT,LKG_NplsPW_cpl_LT,LKG_PplsNW_cpl_LT,OPEN_CACMM1_N_HOLE_D01_cpl_LT,OPEN_CACMM1_P_HOLE_D01_cpl_LT,OPEN_PCCBM1_N_2A_D0X_cpl_8cl_cyld_LT,PBLN302FGC_16FET_res_LT,PBLN304FGC_16FET_res_LT,PBLP302FGC_16FET_res_LT,PBLP304FGC_16FET_res_LT,PCLN2FGC_array_res_LT,PCLP2FGC_array_res_LT,PCOp_ALL_86cl_cyld_LT,PCOp_LN_60_DB_A_Rs_LT,PCOp_LN_60_DB_B_Rs_LT,PCOp_LP_60_MB_A_Rs_LT,PCOp_LP_60_MB_B_Rs_LT,PCOp_RN_60_DB_A_Rs_LT,PCOp_RN_60_DB_B_Rs_LT,PCOp_RP_60_MB_A_Rs_LT,PCOp_RP_60_MB_B_Rs_LT,PCOp_SLN_60_DB_A_Rs_LT,PCOp_SLN_60_DB_B_Rs_LT,PCOp_SLP_60_MB_A_Rs_LT,PCOp_SLP_60_MB_B_Rs_LT,PCOp_ULN_60_DB_A_Rs_LT,PCOp_ULN_60_DB_B_Rs_LT,PCOp_ULP_60_MB_A_Rs_LT,PCOp_ULP_60_MB_B_Rs_LT,PCRN2FGC_array_res_LT,PCRP2FGC_array_res_LT,PCS54LN2FGC_2x2_res_LT,PCS54LPGC_2x2_res_LT,PCSLN2FGC_2x2_res_LT,PCSLNGC_2x2_res_LT,PCSLP2FGC_2x2_res_LT,PCSLP2FGC_array_res_LT,PCSLPGC_2x2_res_LT,PC_RES_LVTN_cpl_LT,PC_RES_LVTP_cpl_LT,PC_RES_ULVTN_cpl_LT,PC_RES_ULVTP_cpl_LT,PC_RES_sLVTN_cpl_LT,PC_RES_sLVTP_cpl_LT,QSTC_CMJ_I_PC_Short_075_cpl_LT,QSTC_CMJ_L_PC_Short_075_cpl_LT,QSTC_CMJ_T_PC_Short_075_cpl_LT,SR_M01_D13_ND2ND_CAyNom_Short_abs_LT,SR_M01_ND2ND_CAyMA_DM7_LT,SR_M01_ND2ND_CAyMA_UM7_

In [81]:
merged_df = w_df.merge(table, how='inner', on=['wafer_id'])

In [82]:
merged_df.to_csv('~/Downloads/justin.csv')